# Ranking results with ORDERed GROUP BYs on a single dimension

__Imagine you’re building a dynamic filter in your app: you want to populate it with, say, the
top most popular (COUNT) dimension values in descending order (ORDER BY). Druid speeds up this
type of query using approximation by default. In this tutorial, work through some examples and
see the effect of turning it off.__

## Prerequisites

* Import the "FlightCarrierOnTime (1 month)" sample date into the default table called `On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11`


In [ ]:
import druidapi
import json
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

# druid_host is the hostname and port for your Druid deployment. 
# In the Docker Compose tutorial environment, this is the Router
# service running at "http://router:8888".
# If you are not using the Docker Compose environment, edit the `druid_host`.

druid_host = "http://router:8888"
druid_host

druid = druidapi.jupyter_client(druid_host)
display = druid.display
sql_client = druid.sql

## Example top-n style queries

Druid looks out for patterns in incoming SQL SELECT statements to work out if it would benefit from
being executed using approximation. A ranking query, like the one below, matches the rules for topn
approximation and in Druid it is enabled by default.

To see this happen, we need an SQL statement that has:
* A GROUP BY on one dimension, and has
* an ORDER BY on one aggregate.

Run this query to see what the results are like:

In [ ]:
sql = '''
SELECT "Reporting_Airline", COUNT(*), SUM("Distance")
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
'''
display.sql(sql)

Using the `explain_sql` method (you could also use EXPLAIN PLAN FOR in native SQL APIs) you can see whether approximation was used:

In [ ]:
sql = '''
SELECT "Reporting_Airline", COUNT(*), SUM("Distance")
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
'''

sql_client.explain_sql(sql)

You know approximation is used when the `queryType` is `topN`.

Druid automatically applies a `LIMIT` operation, not just on the final result set, but
on the results calculated by each server that’s been called upon to answer the query. This results in
less data bubbling up from each process to be merged overall, and therefore the promise of better
efficiency in how this type of query executes.

Notice the `threshold` value? The parallelised `LIMIT` was the `max` of both the `threshold` shown here – which came from the
`LIMIT` in the SQL - and a configuration setting in your cluster – the default for which is 1,000.

You can find out how to read and set this default `LIMIT` in the [documentation](https://druid.apache.org/docs/latest/querying/topnquery.html#aliasing).

As a first step in understanding the implications, we need to find data in our sample set where the
cardinality of the dimension that we will `GROUP BY` exceeds that number. By default, that is 1000.

What's the cardinality of our dimension?

In [ ]:
sql = '''
SELECT COUNT (DISTINCT "Reporting_Airline")
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
'''
display.sql(sql)

This is too low – the initial `LIMIT` has no effect! This means there is no trimming happening anywhere in the database. As a result, as the documentation explains, our results are going to be without error. All the data servers will return all their results, without trimming, to be merged and passed back to us.

Let's find another dimension.

In [ ]:
sql = '''
SELECT COUNT (DISTINCT "Tail_Number")
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
WHERE "Tail_Number" <> ''
'''
display.sql(sql)

With this many distinct values to `GROUP BY`, we know that data servers will trim their results when the
`topN` engine is engaged.

There is another factor to consider – distribution.

In [ ]:
sql = '''
SELECT "Tail_Number", COUNT(*)
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
WHERE "Tail_Number" <> ''
GROUP BY 1
ORDER BY 2 DESC
LIMIT 500
'''

df4 = pd.DataFrame(sql_client.sql(sql))

df4.plot(x='Tail_Number', y='EXPR$1', marker='o')
plt.xticks(rotation=45, ha='right')
plt.gca().get_legend().remove()
plt.show()

Imagine that the cut-off point is in the first 10% of results - in this sample data
that's about the first 400.

The distribution above shows that there is a _very_ high chance that the top result
is going to be top result across all our data, and the second, and the third, and so on.
The same ranking will, very likely, come back from all of the servers.

But as we approach 1000, 25% of the way along, we have a flatter distribution. It is
not as predictable any more where results will rank. Consider, too, that this is a very
simple distribution plot: what will happen when we have `WHERE` on `__time` or other dimensions?

Let's find out what the impact of the initial `LIMIT` is on our results by comparing two result sets:
one with topn enabled, and one with topn disabled. And let's focus on around 10% of the data.

We will run two queries, `sql1` and `sql2`. The only difference will be that we apply the `useApproximateTopN`
query context parameter to turn off approximation for `sql2`.

Then we will use the `compare` method to see whether the results differ.

In [ ]:
sql = '''
SELECT "Tail_Number", COUNT(*) AS "count", SUM(Distance) AS "distance"
    FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
    WHERE "Tail_Number" IS NOT NULL
    GROUP BY 1
    ORDER BY 3 DESC
    LIMIT 500
'''

req = sql_client.sql_request(sql)
req.add_context("useApproximateTopN", "false")
resp = sql_client.sql_query(req)

df1 = pd.DataFrame(sql_client.sql(sql))
df2 = pd.DataFrame(sql_client.sql_query(req).rows)

df3 = df1.compare(df2)
df3

There are two things to notice:

1. Some rows are in different places, and
2. Some values are different

This is because certain data servers returned different sets of results, depending entirely on the local distribution.

* `N829MH` has a different value and position - 43 versus 42.
* `N566JB` has a different rank position because of that – 42 versus 43.

Let's try this with a different dimension that has a different distribution pattern. Let's find another candidate dimension.

In [ ]:
sql = '''
SELECT COUNT(DISTINCT "Flight_Number_Reporting_Airline")
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
WHERE "Flight_Number_Reporting_Airline" <> ''
'''

display.sql(sql)

This is within the default `LIMIT` boundary.

Importantly, it's also greater than the cardinality of the other dimension. It promises to
introduce much greater efficiency in the query execution, and promises better performance!

But before we get too excited, let's check the distribution.

In [ ]:
sql = '''
SELECT "Flight_Number_Reporting_Airline", COUNT(*)
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
WHERE "Flight_Number_Reporting_Airline" <> ''
GROUP BY 1
ORDER BY 2 DESC
LIMIT 500
'''

df5 = pd.DataFrame(sql_client.sql(sql))

df5.plot(x='Flight_Number_Reporting_Airline', y='EXPR$1', kind="bar", xticks=[])
plt.gca().get_legend().remove()
plt.show()

This is a much flatter distribution.

We're much less likely to have the same ranking across the board.

Let's see how this pushed-down, parallelised `LIMIT` operation affects results.

For brevity's sake, let's just look at only the top 10 results. Remember, with `topN`, our own
`LIMIT` is applied on the final result set – but there's still the parallelised `LIMIT`
on each data server.

In [ ]:
sql = '''
SELECT "Flight_Number_Reporting_Airline", AVG("Distance")
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
WHERE "Flight_Number_Reporting_Airline" IS NOT NULL
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
'''

req = sql_client.sql_request(sql)
req.add_context("useApproximateTopN", "false")
resp = sql_client.sql_query(req)

df1 = pd.DataFrame(sql_client.sql(sql))
df2 = pd.DataFrame(sql_client.sql_query(req).rows)

df3 = df1.compare(df2)
df3

Here the impact of a flatter distribution over a greater cardinality is clear,
not just in ranking order, but also in the values that have been calculated
to give us that ranking.

Reporting airline `17` is in a lower position with `topN` than without it. And
the calculation itself, because it non-additive, gives a higher error.

topN is useful for interactive elements, then, like filters or initial lists of results to
deep dive into. That's because of the speed boost we receive at the expense of accuracy –
the mantra for all approximation.

We've seen that the accuracy of the ranking depends greatly on data distribution, and
thereby on what each of the data servers "vote" for in terms of position.

In one final example, let's be more realistic in use. An dynamically-populated list
of filter options is likely to span just a particular period, say two weeks.

Let's set a time period in our query, noting that this will impact (a) the cardinality
of the dimension we `GROUP BY`, and (b) the number of data servers that participate in the query
and "vote" on the rankings.

First, let's look at the cardinality:

In [ ]:
sql = '''
SELECT COUNT (DISTINCT "Tail_Number")
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
WHERE "Tail_Number" <> ''
AND (TIMESTAMP '2005-11-01' <= "__time" AND "__time" <= TIMESTAMP '2005-11-14')
'''
display.sql(sql)

Now we understand that `topN` will affect on the results in this period,
let's plot the distribution.

In [ ]:
sql = '''
SELECT "Tail_Number", COUNT(*)
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
WHERE "Tail_Number" <> ''
AND (TIMESTAMP '2005-11-01' <= "__time" AND "__time" <= TIMESTAMP '2005-11-14')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 500
'''

df4 = pd.DataFrame(sql_client.sql(sql))

df4.plot(x='Tail_Number', y='EXPR$1', marker='o')
plt.xticks(rotation=45, ha='right')
plt.gca().get_legend().remove()
plt.show()

This looks like a fairly good distribution pattern for us to use for our purpose of
an interactive filter.

Let's see how it plays out, accurate versus inaccurate:

In [ ]:
sql = '''
SELECT "Tail_Number", COUNT(*) AS "count", SUM(Distance) AS "distance"
    FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
    WHERE "Tail_Number" IS NOT NULL
    AND (TIMESTAMP '2005-11-01' <= "__time" AND "__time" <= TIMESTAMP '2005-11-14')
    GROUP BY 1
    ORDER BY 3 DESC
    LIMIT 500
'''

req = sql_client.sql_request(sql)
req.add_context("useApproximateTopN", "false")
resp = sql_client.sql_query(req)

df1 = pd.DataFrame(sql_client.sql(sql))
df2 = pd.DataFrame(sql_client.sql_query(req).rows)

df3 = df1.compare(df2)
df3

The distribution, together with our filters, means that our results are very close to accurate.

In conclusion:

* TopN is the default execution model for `GROUP BY` queries with one dimension, an `ORDER BY` and a `LIMIT` clause
* You can turn it off with a query context parameter
* Accuracy is highly dependent on distribution of the data, after filters etc., across the database